In [1]:
import numpy as np
import pandas as pd

from desdeo_problem.testproblems.TestProblems import test_problem_builder

from desdeo_emo.EAs import IOPIS_RVEA, IOPIS_NSGAIII
from desdeo_emo.EAs.NSGAIII import NSGAIII
from desdeo_emo.EAs.RVEA import RVEA
import plotly.express as ex
from pygmo import fast_non_dominated_sorting as nds
import plotly.graph_objects as go

In [2]:
problem = test_problem_builder(name="DTLZ7", n_of_objectives=3, n_of_variables=5)

In [3]:
wholefront = NSGAIII(problem)

In [4]:
while wholefront.continue_evolution():
    wholefront.iterate()

In [5]:
ex.scatter_3d(x=wholefront.population.objectives[:,0],y=wholefront.population.objectives[:,1],z=wholefront.population.objectives[:,2])

In [6]:
problem.ideal = wholefront.population.objectives.min(axis=0)
problem.nadir = wholefront.population.objectives.max(axis=0)

In [7]:
evolver = IOPIS_NSGAIII(problem, n_gen_per_iter=200)

In [8]:
plot, pref = evolver.requests()

In [9]:
print(pref.content)

{'dimensions_data':                 f1        f2      f3
minimize         1         1       1
ideal            0         0  2.6502
nadir     0.908217  0.892777       6, 'message': 'Provide a reference point worse than to the ideal point and better than the nadir point.\nIdeal point: \nf1         0\nf2         0\nf3    2.6502\nName: ideal, dtype: object\nNadir point: \nf1    0.908217\nf2    0.892777\nf3           6\nName: nadir, dtype: object\nThe reference point will be used to create scalarization functions in the preferred region.\n', 'validator': <function validate_ref_point_with_ideal_and_nadir at 0x000001A8DF4548B8>}


In [10]:
response = np.asarray([[0.7,0.1, 4]])
pref.response = pd.DataFrame(response, columns=pref.content['dimensions_data'].columns)

In [11]:
plot, pref = evolver.iterate(pref)

In [12]:
def plot3d(plot, response):
    # IOPISEA
    obj = plot.content['data'].values
    non_dom_id = nds(obj)[0][0]
    fig = go.Figure(
        go.Scatter3d(
            x=obj[non_dom_id, 0],
            y=obj[non_dom_id, 1],
            z=obj[non_dom_id, 2],
            name="IOPIS-RVEA",
            mode='markers',
            marker=dict(
            size=4,
            opacity=0.8
        )
        )
    )

    #True Front
    fig.add_trace(
        go.Scatter3d(
            x=wholefront.population.objectives[:,0],
            y=wholefront.population.objectives[:,1],
            z=wholefront.population.objectives[:,2],
            name="True Front",
            mode="markers",
            marker=dict(
            size=4,
            opacity=0.8
        )
        )
    )

    #Nadir Point
    nadir = wholefront.population.objectives.max(axis=0)
    fig.add_trace(
        go.Scatter3d(
            x=[nadir[0]],
            y=[nadir[1]],
            z=[nadir[2]],
            name="Nadir Point",
            marker=dict(
            size=4,
            opacity=0.8
        )
        )
    )

    # Ideal Point
    ideal = wholefront.population.objectives.min(axis=0)
    fig.add_trace(
        go.Scatter3d(
            x=[ideal[0]],
            y=[ideal[1]],
            z=[ideal[2]],
            name="Ideal Point",
            marker=dict(
            size=4,
            opacity=0.8
        )
        )
    )


    # Reference Point

    fig.add_trace(
        go.Scatter3d(
            x=[response[0,0]],
            y=[response[0,1]],
            z=[response[0,2]],
            name="Reference Point",
            marker=dict(
            size=4,
            opacity=0.8
        )
        )
    )

    # STOM
    lam = 10
    fig.add_trace(
        go.Scatter3d(
            x=[lam * ideal[0] + (1-lam) * response[0,0], response[0,0], (1-lam) * ideal[0] + lam*response[0,0]],
            y=[lam * ideal[1] + (1-lam) * response[0,1], response[0,1], (1-lam) * ideal[1] + lam*response[0,1]],
            z=[lam * ideal[2] + (1-lam) * response[0,2], response[0,2], (1-lam) * ideal[2] + lam*response[0,2]],
            name="STOM",
            marker=dict(
            size=2,
            opacity=0.8
        ),
            line=dict(width=10)
        )
    )

    #AUG

    fig.add_trace(
        go.Scatter3d(
            x=[lam * nadir[0] + (1-lam) * response[0,0], response[0,0], (1-lam) * nadir[0] + lam*response[0,0]],
            y=[lam * nadir[1] + (1-lam) * response[0,1], response[0,1], (1-lam) * nadir[1] + lam*response[0,1]],
            z=[lam * nadir[2] + (1-lam) * response[0,2], response[0,2], (1-lam) * nadir[2] + lam*response[0,2]],
            name="AUG",
            marker=dict(
            size=2,
            opacity=0.8
        ),
            line=dict(width=10)
        )
    )


    fig.update_layout(
        scene = dict(
            xaxis = dict(range=[0,1],),
            yaxis = dict(range=[0,1],),
            zaxis = dict(range=[2,6.1],),
            aspectratio=dict(x=1,y=1,z=1))
                     )
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    return fig
    

In [13]:
fig = plot3d(plot, response)
fig